In [5]:
import pandas as pd
import time
import random
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm
from unidecode import unidecode

### Funciones básicas para realizar scraping
#### Funciones para limpiar texto y moneda

In [6]:
#Función para obtener tipo de cambio
def usd(token):
    #Obtener tipo de cambio
    banxico="https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos/?token="+token
    r=requests.get(banxico).json()
    #Obtener último dato
    mxn=r["bmx"]["series"][0]["datos"][-1]["dato"]
    #transformar a float
    mxn=float(mxn)
    return mxn

In [7]:
#Leer token de banxico
with open("C:/token_banxico.txt", "r") as file:
    llave = file.read().strip()

In [8]:
#Almacenar el tipo de cambio del día
tipo_cambio=usd(llave)
#Imprimir tipo de cambio y fecha
print(f"Hoy es {time.strftime('%d/%m/%Y')} y el tipo de cambio es {tipo_cambio}")

Hoy es 27/09/2025 y el tipo de cambio es 18.3825


In [9]:
def limpia_texto(text):
    if text is None:
        return ""
    # Elimina caracteres no alfanuméricos, caracteres, puntuación, espacios extras y signos de pesos
    cleaned_text = re.sub(r'[^\w\s.]', '', text).strip()
    # Minúsculas
    cleaned_text = cleaned_text.lower()
    #Eliminar acentos
    cleaned_text = unidecode(cleaned_text)
    return cleaned_text

def limpia_moneda(text):
    if text is None:
        return ""
    #Eliminar "\n"
    cleaned_coin = re.sub(r'\n', '', text).strip()
    #Elimina comas
    cleaned_coin = re.sub(r',', '', text).strip()
    #Eliminar signo de pesos
    cleaned_coin = re.sub(r'$', '', cleaned_coin)

    return cleaned_coin

In [10]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}



def equalize_lists(main_list, *lists):
    main_length = len(main_list)
    for lst in lists:
        while len(lst) < main_length:
            lst.append(None)

In [11]:
def pincali(estado,tipo="venta", tc=None):
    if tipo == "venta":
        base_url = "https://www.pincali.com/inmuebles/propiedades-residenciales-en-venta-en-{}?page={}"
    elif tipo == "renta":
        base_url = "https://www.pincali.com/inmuebles/propiedades-residenciales-en-renta-en-{}?page={}"
    elif tipo == "terreno":
        base_url = "https://www.pincali.com/inmuebles/terrenos-en-venta-en-{}-{}?page={}"
    else:
        raise ValueError("Selecciona un tipo de propiedad válido: venta, renta o terreno")
    

    all_data_frames = []

    for page_num in tqdm(range(1, 50), desc=f"Scrapeando Pincali en {estado}"):
        url = base_url.format(estado,page_num)
        r = requests.get(url, headers=headers)
        sopa = BeautifulSoup(r.text, 'html.parser')

        # Cosas a obtener
        recamaras, bathrooms, superficie, direcciones, ofertas, precios, latitud, longitud = [], [], [], [], [], [], [], []

        for precio in sopa.find_all('li', class_='price'):
            precios.append(precio.text.strip())
        for coord in sopa.find_all('li', class_='property__component'):
            latitud.append(coord.get('data-lat'))
            longitud.append(coord.get('data-long'))
        for div in sopa.find_all('div', class_='features'):
            match = re.search(r'(\d+)\s*recámaras', div.text)
            recamaras.append(int(match.group(1)) if match else None)
            match = re.search(r'(\d+)\s*baños', div.text)
            bathrooms.append(int(match.group(1)) if match else None)
            match = re.search(r'(\d+\.?\d*)\s*m²', div.text)
            superficie.append(float(match.group(1)) if match else None)
        for element in sopa.find_all('a', class_='property__content property-content'):
            direcciones.append(element.find('div', class_='location').text.strip())
        for title in sopa.find_all('div', class_='title'):
            ofertas.append(title.text.strip())

                # Filtrar precios según el tipo de propiedad
        if tipo in ["venta", "terreno"]:
            precios = [price for price in precios if "En Renta" not in price]
        elif tipo == "renta":
            precios = [price for price in precios if "En Venta" not in price]

        equalize_lists(ofertas, recamaras, bathrooms, superficie, direcciones, precios, latitud, longitud)

        data_frame = pd.DataFrame({
            'oferta': ofertas, 'precio': precios, 'mts': superficie,
            'bathrooms': bathrooms, 'recamaras': recamaras, 'lat': latitud,
            'lon': longitud, 'fuente': 'easyaviso'
        })
        all_data_frames.append(data_frame)
    combined_df = pd.concat(all_data_frames, ignore_index=True)
    
    if combined_df.empty:
        return combined_df
    
    if tipo in ["venta", "terreno"]:
        combined_df["precio"] =(combined_df["precio"].apply(limpia_moneda)
                                .str.replace("$", "", regex=False)
                                .str.replace("Consulte precio", "0", regex=False)
                                .str.replace("En Venta", "", regex=False)
                                .str.replace("\n","", regex=False))
    elif tipo == "renta":
        combined_df["precio"] =(combined_df["precio"].apply(limpia_moneda)
                                .str.replace("$", "", regex=False)
                                .str.replace("Consulte precio", "0", regex=False)
                                .str.replace("En Renta", "", regex=False)
                                .str.replace("\n","", regex=False))
    combined_df = combined_df[~combined_df["precio"].str.contains("por m²")]
    combined_df = combined_df[~combined_df["precio"].str.contains("por ha")]
    combined_df["precio"] = combined_df["precio"].apply(lambda x: float(x.replace("US", "")) *tc if "US" in x else x)
    combined_df["precio"] = pd.to_numeric(combined_df["precio"], errors="coerce")
    #Eliminar nans
    combined_df = combined_df[combined_df["precio"].notna()]
        #Precio a float
    combined_df["precio"] = combined_df["precio"].astype(float)
        #Eliminar "\n" de precio
    combined_df = combined_df[combined_df["precio"] != 0]
    # Añadir fecha de consulta
    combined_df["fecha_consulta"] = pd.to_datetime("today")
    #Añadir fuente
    combined_df["fuente"] = "easyaviso"
    #Limpiar oferta
    combined_df["oferta"] = combined_df["oferta"].apply(limpia_texto)
    return combined_df

In [12]:
def limpia_datos(df):
    df = df.reset_index(drop=True)
   
    #Eliminar registros con precio 0 o nan
    df=df[df['precio']>0]
    df=df[df['precio'].notna()]
    #Eliminar registros que en oferta contengan "terreno"
    df=df[~df['oferta'].str.contains('terreno')]
    df=df[~df['oferta'].str.contains('remodelar')]
    df=df[~df['oferta'].str.contains('hectareas')]
    #Si la fuente es goodlers, sacar el promedio de precio_min y precio_max y ponerlo en precio
    #Eliminar registros con misma oferta y mismo precio
    df=df.drop_duplicates(subset=['oferta','precio','recamaras','bathrooms'],keep='first')
    #Calcular precio por metro cuadrado
    df['precio_m2'] = df['precio'] / df['mts']

    return df

In [13]:
#Venta
vivi_venta=pincali("ensenada-baja-california","venta", tc=tipo_cambio)

Scrapeando Pincali en ensenada-baja-california: 100%|██████████| 49/49 [00:45<00:00,  1.09it/s]


In [16]:
vivi_limpia_venta=vivi_venta.copy()
#Eliminar si oferta dice "lote" o "terreno"
vivi_limpia_venta=vivi_limpia_venta[~vivi_limpia_venta["oferta"].str.contains("lote|terreno|renta")]
#Eliminar si oferta es ubicacion, precio u operacion
vivi_limpia_venta=vivi_limpia_venta[~vivi_limpia_venta["oferta"].str.contains("ubicacion|precio|operacion")]
#Eliminar si lat es nulo
vivi_limpia_venta=vivi_limpia_venta[vivi_limpia_venta['lat'].notna()]
#Aplicar función de limpieza
vivi_limpia_venta=limpia_datos(vivi_limpia_venta)
vivi_limpia_venta

,oferta,precio,mts,bathrooms,recamaras,lat,lon,fuente,fecha_consulta,precio_m2
0,casa nueva independiente modelo monaco,3308850.0,110.00,NaN,2.0,31.965445,-116.646471,easyaviso,2025-09-27 22:24:19.403530,30080.454545
1,vive o invierte propiedad lista con proyecto t...,1211.0,NaN,NaN,NaN,31.8773451,-116.6289596,easyaviso,2025-09-27 22:24:19.403530,NaN
2,zevze,5239012.5,135.00,2.0,2.0,32.0557002,-116.8794595,easyaviso,2025-09-27 22:24:19.403530,38807.500000
3,casa en venta modelo azur vinedos del mar ens...,250000.0,200.00,NaN,NaN,31.8780559,-116.5377134,easyaviso,2025-09-27 22:24:19.403530,1250.000000
4,casa en venta en fracc. buenaventura en ensena...,4130000.0,139.93,2.0,NaN,31.8395089,-116.6097319,easyaviso,2025-09-27 22:24:19.403530,29514.757379
...,...,...,...,...,...,...,...,...,...,...
564,proyecto mm ranch,8569000.0,172.00,2.0,2.0,31.8775715318,-116.6222977491,easyaviso,2025-09-27 22:24:19.403530,49819.767442
565,departamento centrico en venta modernos amenid...,9800000.0,282.00,3.0,3.0,31.867649,-116.6276616,easyaviso,2025-09-27 22:24:19.403530,34751.773050
566,fracc. residencial andarez,3950000.0,160.00,4.0,4.0,31.8902937169,-116.6120493264,easyaviso,2025-09-27 22:24:19.403530,24687.500000
567,departamento en venta nuevo centrico pet friendly,3500000.0,73.00,2.0,2.0,31.9146405726,-116.6916496343,easyaviso,2025-09-27 22:24:19.403530,47945.205479


In [17]:
#guardar csv
vivi_limpia_venta.to_csv("viviendas_pincali_venta.csv",index=False)